# Check pnl across trades and positions

* Compare trade pnl between two subgraphs
* Compare position pnl between two subgraphs


In [1]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from web3 import Web3
from web3.middleware import geth_poa_middleware

load_dotenv()

pio.renderers.default = 'notebook'


In [2]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
OLD_SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-perps'
SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/tburm/optimism-perps'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'

# get a web3 provider
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [3]:
# data contract
PerpsV2DataAbi = """
[
    {
      "inputs": [
        {
          "internalType": "contract IAddressResolver",
          "name": "_resolverProxy",
          "type": "address"
        }
      ],
      "payable": false,
      "stateMutability": "nonpayable",
      "type": "constructor"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "allMarketSummaries",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "allProxiedMarketSummaries",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "globals",
      "outputs": [
        {
          "components": [
            {
              "internalType": "uint256",
              "name": "minInitialMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationFeeRatio",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationBufferRatio",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "minKeeperFee",
              "type": "uint256"
            }
          ],
          "internalType": "struct PerpsV2MarketData.FuturesGlobals",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "contract IPerpsV2MarketViews",
          "name": "market",
          "type": "address"
        }
      ],
      "name": "marketDetails",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "baseAsset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "marketKey",
              "type": "bytes32"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxLeverage",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "maxMarketValue",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketLimits",
              "name": "limits",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxFundingVelocity",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "skewScale",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FundingParameters",
              "name": "fundingParameters",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "marketSize",
                  "type": "uint256"
                },
                {
                  "components": [
                    {
                      "internalType": "uint256",
                      "name": "long",
                      "type": "uint256"
                    },
                    {
                      "internalType": "uint256",
                      "name": "short",
                      "type": "uint256"
                    }
                  ],
                  "internalType": "struct PerpsV2MarketData.Sides",
                  "name": "sides",
                  "type": "tuple"
                },
                {
                  "internalType": "uint256",
                  "name": "marketDebt",
                  "type": "uint256"
                },
                {
                  "internalType": "int256",
                  "name": "marketSkew",
                  "type": "int256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketSizeDetails",
              "name": "marketSizeDetails",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "price",
                  "type": "uint256"
                },
                {
                  "internalType": "bool",
                  "name": "invalid",
                  "type": "bool"
                }
              ],
              "internalType": "struct PerpsV2MarketData.PriceDetails",
              "name": "priceDetails",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32",
          "name": "marketKey",
          "type": "bytes32"
        }
      ],
      "name": "marketDetailsForKey",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "baseAsset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "marketKey",
              "type": "bytes32"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxLeverage",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "maxMarketValue",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketLimits",
              "name": "limits",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxFundingVelocity",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "skewScale",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FundingParameters",
              "name": "fundingParameters",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "marketSize",
                  "type": "uint256"
                },
                {
                  "components": [
                    {
                      "internalType": "uint256",
                      "name": "long",
                      "type": "uint256"
                    },
                    {
                      "internalType": "uint256",
                      "name": "short",
                      "type": "uint256"
                    }
                  ],
                  "internalType": "struct PerpsV2MarketData.Sides",
                  "name": "sides",
                  "type": "tuple"
                },
                {
                  "internalType": "uint256",
                  "name": "marketDebt",
                  "type": "uint256"
                },
                {
                  "internalType": "int256",
                  "name": "marketSkew",
                  "type": "int256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketSizeDetails",
              "name": "marketSizeDetails",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "price",
                  "type": "uint256"
                },
                {
                  "internalType": "bool",
                  "name": "invalid",
                  "type": "bool"
                }
              ],
              "internalType": "struct PerpsV2MarketData.PriceDetails",
              "name": "priceDetails",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "address[]",
          "name": "markets",
          "type": "address[]"
        }
      ],
      "name": "marketSummaries",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32[]",
          "name": "marketKeys",
          "type": "bytes32[]"
        }
      ],
      "name": "marketSummariesForKeys",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32",
          "name": "marketKey",
          "type": "bytes32"
        }
      ],
      "name": "parameters",
      "outputs": [
        {
          "components": [
            {
              "internalType": "uint256",
              "name": "takerFee",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "makerFee",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "overrideCommitFee",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "takerFeeDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "makerFeeDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "takerFeeOffchainDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "makerFeeOffchainDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxMarketValue",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxFundingVelocity",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "skewScale",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "nextPriceConfirmWindow",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "delayedOrderConfirmWindow",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "minDelayTimeDelta",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxDelayTimeDelta",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "offchainDelayedOrderMinAge",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "offchainDelayedOrderMaxAge",
              "type": "uint256"
            },
            {
              "internalType": "bytes32",
              "name": "offchainMarketKey",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "offchainPriceDivergence",
              "type": "uint256"
            }
          ],
          "internalType": "struct IPerpsV2MarketSettings.Parameters",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "contract IPerpsV2MarketViews",
          "name": "market",
          "type": "address"
        },
        {
          "internalType": "address",
          "name": "account",
          "type": "address"
        }
      ],
      "name": "positionDetails",
      "outputs": [
        {
          "components": [
            {
              "components": [
                {
                  "internalType": "uint64",
                  "name": "id",
                  "type": "uint64"
                },
                {
                  "internalType": "uint64",
                  "name": "lastFundingIndex",
                  "type": "uint64"
                },
                {
                  "internalType": "uint128",
                  "name": "margin",
                  "type": "uint128"
                },
                {
                  "internalType": "uint128",
                  "name": "lastPrice",
                  "type": "uint128"
                },
                {
                  "internalType": "int128",
                  "name": "size",
                  "type": "int128"
                }
              ],
              "internalType": "struct IPerpsV2MarketBaseTypes.Position",
              "name": "position",
              "type": "tuple"
            },
            {
              "internalType": "int256",
              "name": "notionalValue",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "profitLoss",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "accruedFunding",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "remainingMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "accessibleMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationPrice",
              "type": "uint256"
            },
            {
              "internalType": "bool",
              "name": "canLiquidatePosition",
              "type": "bool"
            }
          ],
          "internalType": "struct PerpsV2MarketData.PositionData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32",
          "name": "marketKey",
          "type": "bytes32"
        },
        {
          "internalType": "address",
          "name": "account",
          "type": "address"
        }
      ],
      "name": "positionDetailsForMarketKey",
      "outputs": [
        {
          "components": [
            {
              "components": [
                {
                  "internalType": "uint64",
                  "name": "id",
                  "type": "uint64"
                },
                {
                  "internalType": "uint64",
                  "name": "lastFundingIndex",
                  "type": "uint64"
                },
                {
                  "internalType": "uint128",
                  "name": "margin",
                  "type": "uint128"
                },
                {
                  "internalType": "uint128",
                  "name": "lastPrice",
                  "type": "uint128"
                },
                {
                  "internalType": "int128",
                  "name": "size",
                  "type": "int128"
                }
              ],
              "internalType": "struct IPerpsV2MarketBaseTypes.Position",
              "name": "position",
              "type": "tuple"
            },
            {
              "internalType": "int256",
              "name": "notionalValue",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "profitLoss",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "accruedFunding",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "remainingMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "accessibleMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationPrice",
              "type": "uint256"
            },
            {
              "internalType": "bool",
              "name": "canLiquidatePosition",
              "type": "bool"
            }
          ],
          "internalType": "struct PerpsV2MarketData.PositionData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "resolverProxy",
      "outputs": [
        {
          "internalType": "contract IAddressResolver",
          "name": "",
          "type": "address"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    }
  ]
"""

PerpsV2DataAddress = "0xF7D3D05cCeEEcC9d77864Da3DdE67Ce9a0215A9D"


In [4]:
# functions
def convertDecimals(x): return Decimal(x) / Decimal(10**18)


def convertBytes(x): return bytearray.fromhex(
    x[2:]).decode().replace('\x00', '')


def clean_df(df, decimal_cols=[], bytes_cols=[]):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    for col in bytes_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertBytes)
        else:
            print(f"{col} not in DataFrame")
    return df


async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df


async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True

    df = pd.DataFrame(all_results)
    return df


In [5]:
transfer_query = gql("""
query transfers(
    $last_id: ID!
  ) {
		futuresMarginTransfers(
      where: {
        id_gt: $last_id
      }
			first: 1000
		) {
			id
			timestamp
			account
			market
			size
			asset
			txHash
		}
}
""")
                     
trade_query = gql("""
query trades(
    $last_id: ID!
  ) {
    futuresTrades(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      timestamp
      asset
      marketKey
      orderType
      price
      margin
      positionId
      feesPaid
      pnl
      size
      positionSize
      positionClosed
      trackingCode
    }  
}
""")


## Get transfers and trades

In [6]:
trade_params = {
    'last_id': ''
}

trade_decimal_cols = [
    'price',
    'margin',
    'feesPaid',
    'pnl',
    'size',
    'positionSize',
]

trade_bytes_cols = ['asset']

df_trade = await run_recursive_query(trade_query, trade_params, 'futuresTrades')
df_trade = df_trade.replace({None: np.nan})
df_trade = clean_df(df_trade, decimal_cols=trade_decimal_cols, bytes_cols=trade_bytes_cols)

In [7]:
transfer_params = {
    'last_id': ''
}

transfer_cols = [
    'size',
]

transfer_bytes_cols = [
    'asset'
]

df_transfer = await run_recursive_query(transfer_query, transfer_params, 'futuresMarginTransfers')
df_transfer = df_transfer.replace({None: np.nan})
df_transfer = clean_df(df_transfer, decimal_cols=transfer_cols, bytes_cols=transfer_bytes_cols)


In [8]:
## summarize transfers
df_transfer_summ = df_transfer.groupby('asset')['size'].sum().reset_index()
df_transfer_summ.columns = ['asset', 'netTransfers']

## summarize trades
df_trade_summ = df_trade.groupby('asset')['feesPaid'].sum().reset_index()
df_trade_summ.columns = ['asset', 'feesPaid']


In [9]:
df_trade_summ

,asset,feesPaid
0,AAVE,3993.378385760098038307
1,APE,19624.204403475538214628
2,ATOM,3526.082922839817751173
3,AUD,31.299906023643989742
4,AVAX,30385.775410069270688288
5,AXS,255.928691860156036022
6,BNB,22993.964067132997733252
7,DOGE,6400.357453302724853725
8,DYDX,27824.012797639992069418
9,EUR,1740.413677519895871752


## Get the market data

In [10]:
perpsV2Data = w3.eth.contract(address=PerpsV2DataAddress, abi=PerpsV2DataAbi)
marketSummaries = perpsV2Data.functions.allProxiedMarketSummaries().call()



In [11]:
markets = [{
    'asset': market[1].decode().replace('\x00', ''),
    'marketDebt': w3.fromWei(market[7], unit='ether')
} for market in marketSummaries]

df_markets = pd.DataFrame(markets)
df_markets

,asset,marketDebt
0,sETH,6815128.440027810819542369
1,sBTC,6438160.557665422554771209
2,LINK,369085.807959662759656602
3,SOL,209224.818746116718413188
4,AVAX,296949.309112707661712518
5,AAVE,38453.140406892718008506
6,UNI,138298.723485259093390367
7,MATIC,437796.350829536676545389
8,APE,107886.348865449653839976
9,DYDX,113517.613125118211319451


In [12]:
df_pnl = df_markets.merge(df_transfer_summ, on='asset').merge(df_trade_summ, on='asset')
df_pnl['netPnl'] = df_pnl['marketDebt'] - df_pnl['netTransfers'] - df_pnl['feesPaid']
df_pnl

,asset,marketDebt,netTransfers,feesPaid,netPnl
0,sETH,6815128.440027810819542369,7294183.655113492766004773,457359.486359216988821148,-936414.701444898935283552
1,sBTC,6438160.557665422554771209,6552356.250077278647134557,134973.805545678530092683,-249169.497957534622456031
2,LINK,369085.807959662759656602,398321.447801105248958373,46306.014431764755563504,-75541.654273207244865275
3,SOL,209224.818746116718413188,279492.336977039901639227,37685.512503537423457439,-107953.030734460606683478
4,AVAX,296949.309112707661712518,219799.794850603069002004,30385.775410069270688288,46763.738852035322022226
5,AAVE,38453.140406892718008506,39158.332461202791815741,3993.378385760098038307,-4698.570440070171845542
6,UNI,138298.723485259093390367,146675.076027165428784313,6923.344749548542310626,-15299.697291454877704572
7,MATIC,437796.350829536676545389,459065.884887338440660303,40380.451378740597045435,-61649.985436542361160349
8,APE,107886.348865449653839976,95324.878558630093577706,19624.204403475538214628,-7062.734096655977952358
9,DYDX,113517.613125118211319451,138089.309987072462772567,27824.012797639992069418,-52395.709659594243522534


In [13]:
df_pnl.to_csv('output/market_upnl.csv', index=False)

In [14]:
df_pnl['netPnl'].sum()

Decimal('-1761543.096472689743528744')

In [17]:
df_pnl['netPnlClean'] = df_pnl['netPnl'].apply(lambda x: round(x, 2))
df_pnl[['asset', 'netPnlClean']].sort_values('netPnlClean', ascending=True)

,asset,netPnlClean
0,sETH,-936414.70
1,sBTC,-249169.50
11,OP,-231784.19
3,SOL,-107953.03
2,LINK,-75541.65
7,MATIC,-61649.99
9,DYDX,-52395.71
19,FTM,-38605.78
14,XAG,-26801.73
6,UNI,-15299.70
